In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Load Data
df = pd.read_csv('datasets/FilteredData.csv')
df = df[['ClientID','CycleNumber','LengthofCycle']]
df.loc[:, 'LengthofCycle'] = df['LengthofCycle'] + 1

# Find the partition to different ClientID's
change_index = df.index[df['ClientID'] != df['ClientID'].shift()].to_list()
change_index.append(len(df))
change_indexes = [(change_index[i-1], change_index[i]) for i in range(1,len(change_index))]

In [3]:
# helper functions

def calculate_rmse(actual, predicted):
    mask = ~np.isnan(predicted)
    return np.sqrt(np.nanmean((actual[mask] - predicted[mask]) ** 2))

def custom_round(x):
    if x % 1 == 0.5:
        return int(x + 1)
    else:
        return round(x)

Mooving Average (MA)

In [4]:
# Finding the best Time Period to average
max_cycles_to_average = 10
best_rmse = float('inf')
best_parameter = None
column_names = []

for cycles_to_average in range(1,max_cycles_to_average+1):
    column_name = 'MA_'+str(cycles_to_average)
    column_names.append(column_name)
    # df[column_name] = df['LengthofCycle']
    df[column_name] = np.nan

    for start, finish in change_indexes:
        # df.at[start, column_name] = np.nan
        i = start + 1
        while i < finish:
            if df.at[i, 'CycleNumber'] <= cycles_to_average:
                df.at[i, column_name] = df.loc[start:i-1,'LengthofCycle'].mean().round()
                # df.at[i, column_name] = custom_round(df.loc[start:i-1,'LengthofCycle'].mean())
                i += 1
            else:
                df.at[i, column_name] = df.loc[i-cycles_to_average:i-1,'LengthofCycle'].mean().round()
                # df.at[i, column_name] = custom_round(df.loc[i-cycles_to_average:i-1,'LengthofCycle'].mean())
                i += 1

RMSE_list = []
for column_name in column_names:
    rmse = calculate_rmse(actual=df['LengthofCycle'],predicted=df[column_name])
    if rmse < best_rmse:
        best_rmse = rmse
        best_parameter = column_name[-1]
        if best_parameter == '0':
            best_parameter = 10
    RMSE_list.append(rmse)

print("RMSE list:")
print(RMSE_list)
print("Best MA is: ", best_parameter)
print("Best RMSE is: ", best_rmse)

RMSE list:
[3.9502476915394222, 3.4309747512525637, 3.2459531264582897, 3.1716229600248993, 3.132518809952707, 3.117477017180117, 3.1126706640685016, 3.1161364550810733, 3.110432620448637, 3.1117756397773886]
Best MA is:  9
Best RMSE is:  3.110432620448637


Exponential Smoothing (EMA)

In [69]:
# Finding the best Smoothing Factor

def exponential_smoothing_rmse(alpha, actual=df['LengthofCycle'], change_indexes=change_indexes):
    predicted = np.zeros(len(actual))
    for start, finish in change_indexes:
        predicted[start] = np.nan
        predicted[start + 1] = actual[start]
        for i in range(start + 2, finish):
            # predicted[i] = custom_round((1 - alpha) * predicted[i - 1] + alpha * actual[i-1]) # leads to Best smoothing factor is 0.269 and RMSE:  3.1387372156955413
            predicted[i] = round((1 - alpha) * predicted[i - 1] + alpha * actual[i-1])
    return calculate_rmse(actual, predicted)


alpha_range = np.linspace(0,1,1001)

best_alpha = None
best_rmse = float('inf')

for smoothing_factor in alpha_range:
    rmse = exponential_smoothing_rmse(alpha=smoothing_factor)
    if rmse <= best_rmse:
        best_rmse = rmse
        best_alpha = smoothing_factor
print("Best smoothing factor is: ", best_alpha)
print("RMSE: ", best_rmse)

Best smoothing factor is:  0.25
RMSE:  3.1291824379228856


ARMA(1,1)

In [4]:
def ARMA_1_1():
    data = df['LengthofCycle']
    column_name = 'ARMA(1,1)'
    df[column_name] = np.nan
    for start, finish in change_indexes:
        # Initialize parameters
        phi = 0.0  # AR coefficient
        theta = 0.0  # MA coefficient
        for i in range(start + 2, finish):
            # Fit AR model
            ar = data[i - 1] - phi * data[i - 2]

            # Fit MA model
            ma = data[i - 1] - phi * data[i - 2] - theta * ar

            # Combine AR and MA predictions
            df.at[i, column_name] = round(phi * data[i - 1] + theta * ar + ma)

            # Update AR and MA coefficients (you can use different estimation methods)
            phi = (data[i] - data[i - 1]) / data[i - 1]
            theta = (data[i] - data[i - 1]) / ar
        


In [38]:
ARMA_1_1()

In [42]:
# regular rounding: 4.317624536576419,  custom rounding: the same
calculate_rmse(actual=df['LengthofCycle'], predicted=df['ARMA(1,1)'])

4.317624536576419

Combination of MA, EMA, ARMA(1,1)

In [5]:
# MA_9
cycles_to_average = 9
column_name = 'MA_'+str(cycles_to_average)
df[column_name] = np.nan

for start, finish in change_indexes:
    i = start + 1
    while i < finish:
        if df.at[i, 'CycleNumber'] <= cycles_to_average:
            df.at[i, column_name] = df.loc[start:i-1,'LengthofCycle'].mean().round()
            i += 1
        else:
            df.at[i, column_name] = df.loc[i-cycles_to_average:i-1,'LengthofCycle'].mean().round()
            i += 1

# EMA
df['EMA'] = np.nan
alpha = 0.25
for start, finish in change_indexes:
    df.loc[start + 1, 'EMA'] = df.at[start, 'LengthofCycle']
    for i in range(start + 2, finish):
        df.loc[i, 'EMA'] = round((1 - alpha) * df.loc[i - 1, 'EMA'] + alpha * df.at[i - 1, 'LengthofCycle'])

# ARMA(1,1)
ARMA_1_1()

In [8]:
df

,ClientID,CycleNumber,LengthofCycle,MA_9,EMA,"ARMA(1,1)"
0,nfp8122,1,30,NaN,NaN,NaN
1,nfp8122,2,28,30.0,30.0,NaN
2,nfp8122,3,30,29.0,30.0,28.0
3,nfp8122,4,28,29.0,30.0,30.0
4,nfp8122,5,29,29.0,30.0,28.0
...,...,...,...,...,...,...
1549,nfp8334,7,30,30.0,31.0,33.0
1550,nfp8334,8,29,30.0,31.0,30.0
1551,nfp8334,9,29,30.0,30.0,29.0
1552,nfp8334,10,41,30.0,30.0,29.0


In [6]:
# Combining the 3 methods to a single forecast
df['Combined'] = np.nan
for i, r in df.iterrows():
    if df.at[i, 'CycleNumber'] == 1:
        continue
    elif df.at[i, 'CycleNumber'] == 2:
        actual = df.at[i, 'LengthofCycle']
        ma = df.at[i, 'MA_9']
        ema = df.at[i, 'EMA']
        if abs(actual - ma) <= abs(actual - ema):
            df.at[i, 'Combined'] = ma
        else:
            df.at[i, 'Combined'] = ema
    else:
        actual = df.at[i, 'LengthofCycle']
        ma = df.at[i, 'MA_9']
        ema = df.at[i, 'EMA']
        arma = df.at[i, 'ARMA(1,1)']
        if abs(actual - ma) <= abs(actual - ema) and abs(actual - ma) <= abs(actual - arma):
            df.at[i, 'Combined'] = ma
        elif abs(actual - ema) <= abs(actual - ma) and abs(actual - ema) <= abs(actual - arma):
            df.at[i, 'Combined'] = ema
        else:
            df.at[i, 'Combined'] = arma

In [10]:
df

,ClientID,CycleNumber,LengthofCycle,MA_9,EMA,"ARMA(1,1)",Combined
0,nfp8122,1,30,NaN,NaN,NaN,NaN
1,nfp8122,2,28,30.0,30.0,NaN,30.0
2,nfp8122,3,30,29.0,30.0,28.0,30.0
3,nfp8122,4,28,29.0,30.0,30.0,29.0
4,nfp8122,5,29,29.0,30.0,28.0,29.0
...,...,...,...,...,...,...,...
1549,nfp8334,7,30,30.0,31.0,33.0,30.0
1550,nfp8334,8,29,30.0,31.0,30.0,30.0
1551,nfp8334,9,29,30.0,30.0,29.0,29.0
1552,nfp8334,10,41,30.0,30.0,29.0,30.0


In [11]:
calculate_rmse(actual=df['LengthofCycle'], predicted=df['Combined'])

2.6435131094603217

Tradeoff: Number of abstinences to successful forecasts

In [14]:
number_of_abstinences = (
    (df['MA_9'] < df['LengthofCycle']).sum() +
    ((df['EMA'] < df['LengthofCycle']) & (df['EMA'] != df['MA_9'])).sum() +
    ((df['ARMA(1,1)'] < df['LengthofCycle']) & (df['ARMA(1,1)'] != df['MA_9']) & (df['ARMA(1,1)'] != df['EMA'])).sum()
)

total_number_of_forecasts = (
    (df['MA_9'] != np.nan).sum() +
    ((df['EMA'] != np.nan) & (df['EMA'] != df['MA_9'])).sum() +
    ((df['ARMA(1,1)'] != np.nan) & (df['ARMA(1,1)'] != df['MA_9']) & (df['ARMA(1,1)'] != df['EMA'])).sum()
)

successful_forecasts = len(df[df['LengthofCycle'] == df['Combined']])

ratio = number_of_abstinences / successful_forecasts

print("Correct forecasts:", successful_forecasts)
print("False forecasts before menstruation:", number_of_abstinences)
print("Ratio: {:.4f}".format(ratio))
print("Total number of different forecasts:", total_number_of_forecasts)


Correct forecasts: 451
False forecasts before menstruation: 1122
Ratio: 2.4878
Total number of different forecasts: 3415


In [10]:
df['Combined'].isna().sum()

118

In [12]:
df['Combined'].mode()

0    28.0
Name: Combined, dtype: float64

In [13]:
df['Combined'].describe()

count    1436.000000
mean       30.279944
std         3.079375
min        24.000000
25%        28.000000
50%        30.000000
75%        32.000000
max        43.000000
Name: Combined, dtype: float64